# Consigna
El objetivo de la segunda ejercitación obligatoria es predecir el uso de anticoncepción en base a datos demográficos. 

Ustedes van a bajar los datos, entrenar un modelo basado en regresión logística, optimizarlo, etc., usando las herramientas que vimos en la clase.

Luego, les vamos a pasar datos sin etiquetas y ustedes estimarán la probabilidad de que la etiqueta sea positiva (uso de anticoncepción). 

Finalmente, nosotros usaremos esa información para calcular un AUC y rankearlos de acuerdo a la performance de sus modelos.

Módulos

In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

Cargo el dataset

In [ ]:
drive.mount('/content/drive')
filename = '/content/drive/My Drive/anticoncepcion_indonesia.csv'
df = pd.read_csv(filename)

Mounted at /content/drive


In [ ]:
# Categorías binarias ocupacion_hombre
df = pd.get_dummies(df, drop_first=True, columns=['ocupacion_hombre'])

# Categorías binarias metodo_anticonceptivo: si o no.
df['anticonceptivo'] = 0
df.loc[df['metodo_anticonceptivo'] > 1, 'anticonceptivo'] = 1

# Tiene hijos
df['tiene_hijos'] = 0
df.loc[df['numero_hijos'] > 0, 'tiene_hijos'] = 1

¿Qué grado $n$ y qué penalizador $C$ debo elegir?

In [ ]:
def classifier(X, y, n, c):
  X_temp = X
  for i in np.arange(2,n):
    X_temp = np.concatenate((X_temp,X**i), axis=1)
  X = X_temp
  
  # X = PolynomialFeatures(degree=n).fit_transform(X)
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
  
  scaler = MinMaxScaler()
  scaler.fit(X_train)
  X_train = scaler.transform(X_train)

  scaler = MinMaxScaler()
  scaler.fit(X_test)
  X_test = scaler.transform(X_test)

  reg = LogisticRegression(penalty='l2', class_weight='balanced', C=c, max_iter=10000)
  reg.fit(X_train, y_train)

  auc_train = roc_auc_score(y_train, reg.predict_proba(X_train)[:,1])
  auc_test = roc_auc_score(y_test, reg.predict_proba(X_test)[:,1])

  return auc_train, auc_test

In [ ]:
campos = ['edad_mujer', 'educacion_mujer', 'educacion_hombre', 'numero_hijos', 'religion_mujer', 'mujer_trabaja', 'ocupacion_hombre_2', 'ocupacion_hombre_3', 'ocupacion_hombre_4', 'nivel_vida', 'exposicion_medios', 'tiene_hijos']
X = df[campos].values
y = df['anticonceptivo'].values

In [ ]:
iterations = 400
n_values = np.arange(1,5)
c_values = np.arange(0.2, 60, 2)

mean_auc_train = np.zeros((len(n_values), len(c_values)))
mean_auc_test = np.zeros((len(n_values), len(c_values)))

for xx,n in enumerate(n_values):
  for yy,c in enumerate(c_values):
    print([n,c])
    auc_train_temp = []
    auc_test_temp = []
    for it in np.arange(iterations):
      auc_train, auc_test = classifier(X, y, n, c)
      auc_train_temp.append(auc_train)
      auc_test_temp.append(auc_test)
    mean_auc_train[xx, yy] = np.mean(auc_train_temp)
    mean_auc_test[xx, yy] = np.mean(auc_test_temp)

In [ ]:
import matplotlib.pyplot as plt

c_values = [round(elem, 2) for elem in c_values]

fig, ax = plt.subplots(1,1, figsize=(15,15))  # visualizacion train set
img = ax.imshow(mean_auc_train) # graficamos la matriz
y_label_list = n_values # estos son los ticks de los ejes
x_label_list = c_values
ax.set_xticks(np.arange(len(c_values))) # creo los ticks de los ejes
ax.set_yticks(np.arange(len(n_values)))
ax.set_yticklabels(y_label_list) # pongo los ticks de los ejes
ax.set_xticklabels(x_label_list)
plt.xlabel('C (constante de regularizacion)', fontsize=15) # le pongo nombre a ejes y titulo 
plt.ylabel('n (potencia maxima para elevar)', fontsize=15)
plt.title('AUC (train set)', fontsize=20)
fig.colorbar(img, orientation="horizontal", pad=0.2) # la barra de colores donde no me moleste

fig, ax = plt.subplots(1,1, figsize=(15,15)) # visualizacion test set, ver comentarios para el caso train, es idéntico
img = ax.imshow(mean_auc_test)
y_label_list = n_values
x_label_list = c_values
ax.set_xticks(np.arange(len(c_values)))
ax.set_yticks(np.arange(len(n_values)))
ax.set_yticklabels(y_label_list)
ax.set_xticklabels(x_label_list)
plt.xlabel('C (constante de regularizacion)', fontsize=15)
plt.ylabel('n (potencia maxima para elevar)', fontsize=15)
plt.title('AUC (test set)', fontsize=20)
fig.colorbar(img, orientation="horizontal", pad=0.2)

De acá obtengo el $n$ y el $C$ que mejor predicen.

Ahora tengo que buscar el set de train que más información me da. Para eso uso **validación cruzada estratificada**.

In [ ]:
n = 3
c = 56.37

campos = ['edad_mujer', 'educacion_mujer', 'educacion_hombre', 'numero_hijos', 'religion_mujer', 'mujer_trabaja', 'ocupacion_hombre_2', 'ocupacion_hombre_3', 'ocupacion_hombre_4', 'nivel_vida', 'exposicion_medios', 'tiene_hijos']
X = df[campos].values
y = df['anticonceptivo'].values


X_temp = X
for i in np.arange(2,n):
  X_temp = np.concatenate((X_temp,X**i), axis=1)
X = X_temp

skf = StratifiedKFold(n_splits=5, shuffle=True)
skf.get_n_splits(X, y)

auc_values = []
indices = []

for train_index, test_index in skf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    reg = LogisticRegression(penalty = 'l2', class_weight='balanced', C=c, max_iter=10000)
    reg.fit(X_train, y_train)

    probas_test = reg.predict_proba(X_test)
    auc_test = roc_auc_score(y_test, probas_test[:,1])

    auc_values.append(auc_test)
    indices.append(test_index)

print(auc_values)
print(np.mean(auc_values))

[0.7115600448933783, 0.6958041958041958, 0.747086247086247, 0.7412587412587412, 0.7803030303030303]
0.7352024518691185


# Modelo final

In [ ]:
n = 3
c = 56.37

campos = ['edad_mujer', 'educacion_mujer', 'educacion_hombre', 'numero_hijos', 'religion_mujer', 'mujer_trabaja', 'ocupacion_hombre_2', 'ocupacion_hombre_3', 'ocupacion_hombre_4', 'nivel_vida', 'exposicion_medios', 'tiene_hijos']
X = df[campos].values
y = df['anticonceptivo'].values

X_temp = X
for i in np.arange(2,n):
  X_temp = np.concatenate((X_temp,X**i), axis=1)
X = X_temp

scaler = MinMaxScaler()
scaler.fit(X)
X = scaler.transform(X)

reg = LogisticRegression(penalty = 'l2', class_weight='balanced', C=c, max_iter=10000)
reg.fit(X, y)

LogisticRegression(C=56.37, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=10000, multi_class='auto', n_jobs=None,
                   penalty='l2', random_state=None, solver='lbfgs', tol=0.0001,
                   verbose=0, warm_start=False)

In [ ]:
X_test = np.array([20, 3, 3, 0, 0, 0, 0, 0, 0, 3, 1, 0]).reshape(1,-1)
temp = X_test
for i in np.arange(2, n):
  temp = np.concatenate((temp, temp**i), axis=1)
X_test = temp

X_test = scaler.transform(X_test)

In [ ]:
reg.predict_proba(X_test)

array([[0.99879749, 0.00120251]])

In [ ]:
path = '/content/drive/My Drive/LaboDatos2021/anticoncepcion_indonesia_test.csv'
df_test = pd.read_csv(path)
df_test = pd.get_dummies(df_test, drop_first=True, columns=['ocupacion_hombre'])

campos = ['edad_mujer', 'educacion_mujer', 'educacion_hombre', 'numero_hijos','religion_mujer', 'mujer_trabaja', 'ocupacion_hombre_2', 'ocupacion_hombre_3', 'ocupacion_hombre_4', 'nivel_vida', 'exposicion_medios']
X_test = np.array(df_test[campos])

X_temp = X_test
for i in np.arange(2,n):
  X_temp = np.concatenate((X_temp, X_temp**i), axis=1)
X_test = X_temp

scaler = MinMaxScaler()
scaler.fit(X_test)
X_test = scaler.transform(X_test)

clf = LogisticRegression(penalty='l2', class_weight='balanced', C=c, max_iter=10000)
clf.fit(X,y)
probas = clf.predict_proba(X_test)
print(probas)

In [ ]:
save_path= '/content/drive/My Drive/Fernandez_Spandau_Luciana'
np.savetxt(save_path, probas, delimiter=",")

# Resultados
AUC: 0,72346824707.  
Una columna extra que contenga la información binaria acerca de si la mujer tiene hijos o no, hubiera mejorado el resultado.